In [1]:
!pip install -U -q google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.5/111.5 kB 7.4 MB/s eta 0:00:00


In [3]:
from google.colab import userdata
import os

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [4]:
from google import genai
client = genai.Client(http_options= {'api_version': 'v1alpha'})

In [5]:
MODEL = "gemini-2.0-flash-exp"

In [6]:
import asyncio
import base64
import contextlib
import datetime
import os
import json
import wave
import itertools

from IPython.display import display, Audio

from google import genai
from google.genai import types

In [8]:
config={
    "generation_config": {"response_modalities": ["TEXT"]}}

async with client.aio.live.connect(model=MODEL, config=config) as session:
  message = "Hello? Gemini are you there?"
  print("> ", message, "\n")
  await session.send(message, end_of_turn=True)

  # For text responses, When the model's turn is complete it breaks out of the loop.
  turn = session.receive()
  async for chunk in turn:
    if chunk.text is not None:
      print(f'- {chunk.text}')

>  Hello? Gemini are you there? 

- Yes
- , I'm here! How can I help you today?



In [9]:
@contextlib.contextmanager
def wave_file(filename, channels=1, rate=24000, sample_width=2):
    with wave.open(filename, "wb") as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(sample_width)
        wf.setframerate(rate)
        yield wf

In [10]:
config={
    "generation_config": {"response_modalities": ["AUDIO"]}}

async with client.aio.live.connect(model=MODEL, config=config) as session:
  file_name = 'audio.wav'
  with wave_file(file_name) as wav:
    message = "Hello? Gemini are you there?"
    print("> ", message, "\n")
    await session.send(message, end_of_turn=True)

    first = True
    async for response in session.receive():
      if response.data is not None:
        model_turn = response.server_content.model_turn
        if first:
          print(model_turn.parts[0].inline_data.mime_type)
          first = False
        print('.', end='.')
        wav.writeframes(response.data)


display(Audio(file_name, autoplay=True))


>  Hello? Gemini are you there? 

audio/pcm;rate=24000
..............................

In [11]:
import logging

logger = logging.getLogger('Live')
logger.setLevel('INFO')

In [12]:
class AudioLoop:
  def __init__(self, config=None):
    self.session = None
    self.index = 0
    if config is None:
      config={
          "generation_config": {
              "response_modalities": ["AUDIO"]}}
    self.config = config

  async def run(self):
    print("Type 'q' to quit")

    logger.debug('connect')
    async with client.aio.live.connect(model=MODEL, config=self.config) as session:
      self.session = session

      while True:
        # Ideally these would be separate tasks.
        if not await self.send():
          break
        await self.recv()

  async def send(self):
    logger.debug('send')
    # `asyncio.to_thread` is important here, without it all other tasks are blocked.
    text = await asyncio.to_thread(input, "message > ")

    # If the input returns 'q' quit.
    if text.lower() == 'q':
      return False

    # Send the message to the model.
    await self.session.send(text, end_of_turn=True)
    logger.debug('sent')
    return True

  async def recv(self):
    # Start a new `.wav` file.
    file_name = f"audio_{self.index}.wav"
    with wave_file(file_name) as wav:
      self.index += 1

      logger.debug('receive')

      # Read chunks from the socket.
      async for response in self.session.receive():
        logger.debug(f'got chunk: {str(response)}')

        server_content = response.server_content
        if server_content is None:
          logger.error(f'Unhandled server message! - {response}')
          break

        # Write audio the chunk to the `.wav` file.
        model_turn = response.server_content.model_turn

        if model_turn is not None:
          for part in model_turn.parts:
            data = part.inline_data.data
            print('.', end='')
            logger.debug(f'Got pcm_data, mimetype: {part.inline_data.mime_type}')
            wav.writeframes(data)

        if response.server_content.turn_complete:
          print('\n<Turn complete>')
          break

    display(Audio(file_name, autoplay=True))
    await asyncio.sleep(2)


In [ ]:
await AudioLoop().run()

Type 'q' to quit
message > How are you
.................
<Turn complete>


message > Fine! Can you teach me Langgraph ?
.............................................
<Turn complete>


message > Sure! Let's start by making a chatbot for students to help them in Fainancial-Accounting .
............................................
<Turn complete>
